<a href="https://colab.research.google.com/github/pnabende/intelligent-systems-course-unit/blob/main/uncertainty_management_simple_to_moderate_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import package**

In [1]:
import operator

**Define several classes**

In [2]:
class Fact:
    def __init__(self, statement, certainty):
        self.statement = statement.lower()  # Normalize the statement
        self.certainty = certainty

    def __eq__(self, other):
        # This will help in comparing two facts based on their statements.
        return self.statement == other.statement

    def __hash__(self):
        return hash(self.statement)

In [3]:
class Rule:
    def __init__(self, conditions, conclusion, certainty):
        self.conditions = conditions
        self.conclusion = conclusion
        self.certainty = certainty

    def applies(self, facts):
        for cond in self.conditions:
            if cond not in facts:
                return False
        return True

In [4]:
class KnowledgeBase:
    def __init__(self):
        self.rules = []

    def add_rule(self, rule):
        self.rules.append(rule)

class InferenceEngine:
    def __init__(self, knowledge_base):
        self.kb = knowledge_base

    def infer(self, facts):
        conclusions = []
        for rule in self.kb.rules:
            if rule.applies(facts):
                # A simple way to combine certainty factors
                total_certainty = min([fact.certainty for fact in rule.conditions] + [rule.certainty])
                conclusions.append(Fact(rule.conclusion.statement, total_certainty))
        return conclusions



In [5]:
class InferenceEngine:
    def __init__(self, knowledge_base):
        self.kb = knowledge_base

    def infer(self, facts):
        conclusions = []
        for rule in self.kb.rules:
            if rule.applies(facts):
                # Combining certainty: rule's certainty * minimum certainty among rule's conditions
                total_certainty = rule.certainty * min([fact.certainty for fact in facts if fact in rule.conditions])
                conclusions.append(Fact(rule.conclusion.statement, total_certainty))
        return conclusions

In [6]:
class ExpertSystem:
    def __init__(self):
        self.kb = KnowledgeBase()
        self.engine = InferenceEngine(self.kb)

    def add_knowledge(self, rule):
        self.kb.add_rule(rule)

    def assess_situation(self, facts):
        return self.engine.infer(facts)

    def query_system(self):
        # Collect facts from the user
        facts = []
        while True:
            statement = input("Enter a known fact (or 'done' to finish): ")
            if statement == 'done':
                break
            certainty = float(input(f"Enter certainty for '{statement}' (0.0 to 1.0): "))
            facts.append(Fact(statement, certainty))

        # Infer and display conclusions
        conclusions = self.assess_situation(facts)
        if conclusions:
            print("\nConclusions:")
            for conclusion in conclusions:
                print(f"- {conclusion.statement} (certainty: {conclusion.certainty})")
        else:
            print("\nNo conclusions could be drawn from the given facts.")


In [7]:
def main():
    system = ExpertSystem()

    # Define and add some rules to the knowledge base
    rule1 = Rule(
        conditions=[Fact("temperature is high", 0.5), Fact("patient is sweating", 0.5)],  # Reduced certainty requirement
        conclusion=Fact("the patient has a fever", 0.9),
        certainty=0.9
    )
    system.add_knowledge(rule1)

    rule2 = Rule(
        conditions=[Fact("patient has rash", 0.5), Fact("temperature is high", 0.5)],  # Reduced certainty requirement
        conclusion=Fact("the patient has measles", 0.85),
        certainty=0.85
    )
    system.add_knowledge(rule2)

    # Query the system based on user input
    system.query_system()

In [9]:
if __name__ == "__main__":
    main()

Enter a known fact (or 'done' to finish): temperature is high
Enter certainty for 'temperature is high' (0.0 to 1.0): 0.7
Enter a known fact (or 'done' to finish): patient is sweating
Enter certainty for 'patient is sweating' (0.0 to 1.0): 0.5
Enter a known fact (or 'done' to finish): patient has rash
Enter certainty for 'patient has rash' (0.0 to 1.0): 0.2
Enter a known fact (or 'done' to finish): done

Conclusions:
- the patient has a fever (certainty: 0.45)
- the patient has measles (certainty: 0.17)
